In [1]:
import cv2, joblib
import numpy as np

In [ ]:
data = joblib.load("olivetti_pca_svm_simple.joblib")
pca, clf = data['pca'], data['svm']

In [3]:
face_real = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


In [6]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_real.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (64, 64))
        X = face_resized.flatten().reshape(1, -1)
        X_pca = pca.transform(X)
        pred = clf.predict(X_pca)
        conf = np.max(clf.decision_function(X_pca))

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {pred}", (x, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Real-Time Face Recognition (PCA + SVM)", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 